# Исследование надежности заемщиков.

Используя данные о платежеспособности клиентов, будем исследовать, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
#df = pd.read_csv("C:/Users/dimil/OneDrive/Desktop/data.csv")
#display(df.head(10))

In [2]:
#код ревьюера
df = pd.read_csv('/datasets/data.csv')

In [3]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

Первичный обзор вполне достаточен для того, чтобы увидеть целый ряд проблем в представленном наборе данных. 
Так, в столбцах days_employed и total_income остутствует более 2000 значений. 
Похоже, что не все клиенты хотели или могли предоставить данную информацию.
Также уже видно наличие отрицательных значений в столбце days_employed, что противоречит логике.
Наконец, в столбце education видно различное написание одних и тех же слов, что может потребовать предварительной обработки данных для анализа.

### Шаг 2.1 Заполнение пропусков

Сначала определим долю пропусков в каждом из двух столбцов:

In [4]:
void_1 = len(df[df['days_employed'].isna()])
void_2 = len(df[df['total_income'].isna()]) 

print(void_1/len(df['days_employed'])*100,"%")
print(void_2/len(df['total_income'])*100,"%")

10.099883855981417 %
10.099883855981417 %


Процент пропусков довольно высок. Если это возможно, лучше постараться достать отсутствующие данные.

Для того, чтобы продолжить анализ данных, необходимо заполнить недостающие значения. 
В данном случае наиболее походящим для этого является медианное значение. Оно не нарушит распределение данных внутри датасета, как могло бы произойти при заполнении средним значением.

In [5]:
days_median = df['days_employed'].median()
income_median = df['total_income'].median()
print(days_median)
print(income_median)


-1203.369528770489
145017.93753253992


Теперь заполним пропуски найденными значениями:

In [6]:
df['days_employed'] = df['days_employed'].fillna(days_median)
df['total_income'] = df['total_income'].fillna(income_median)


Наконец, убедимся в том, что пропущенных значений в таблице больше нет:

In [7]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

### Шаг 2.2 Проверка данных на аномалии и исправления.

В столбце days_employed видны отрицательные значения. 
Вероятно, это связано с неправильным вводом данных клиента или с ошибкой системы, неправильно отображающей введенные значения. 
Можно предположить и неверное понимание отрицательного значения в данном столбце. Что, если это отображает количество дней без работы?
Если есть возможность уточнения данных для их правильного интерпретирования, лучше всегда сделать это перед дальнейшим анализом.

Допустим, что данные просто отображаются с минусом. Тогда просто избавимся от него:

In [8]:
def negative_num(number):
    if number<0:
        return -number
    else:
        return number


df['days_employed'] = df['days_employed'].apply(negative_num)
display(df.head(10))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверим остальные столбцы на наличие аномалий:

In [9]:

display(df['children'].value_counts())
display(df['income_type'].value_counts())
display(df['dob_years'].value_counts())
display(df['education_id'].value_counts())
display(df['family_status'].value_counts())
display(df['family_status_id'].value_counts())
display(df['gender'].value_counts())
display(df['debt'].value_counts())
display(df['total_income'].value_counts())
display(df['purpose'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

0    19784
1     1741
Name: debt, dtype: int64

145017.937533    2175
112874.418757       1
104381.857170       1
182036.676828       1
122421.963500       1
                 ... 
133299.194693       1
115080.782380       1
84896.781597        1
153838.839212       1
150014.128510       1
Name: total_income, Length: 19351, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Похоже, что в некоторых других столбцах также имеются неверные или неадекватные значения. Сначала исправим столбец children.
Поместим значения 20 и -1 в новую категорию "количеcтво неизвестно":

In [10]:
df.loc[df['children'] == -1,'children'] = "количество неизвестно" 
df.loc[df['children'] == 20,'children'] = "количество неизвестно" 
display(df['children'].value_counts())

0                        14149
1                         4818
2                         2055
3                          330
количество неизвестно      123
4                           41
5                            9
Name: children, dtype: int64

Теперь поработаем со столбцом dob_years, где у 101 человека возраст равен нулю!.
Поместим значения 0 в новую категорию "возраст не указан":

In [11]:
df.loc[df['dob_years'] == 0,'dob_years'] = "возраст не указан" 
display(df['dob_years'].value_counts())

35                   617
40                   609
41                   607
34                   603
38                   598
42                   597
33                   581
39                   573
31                   560
36                   555
44                   547
29                   545
30                   540
48                   538
37                   537
50                   514
43                   513
32                   510
49                   508
28                   503
45                   497
27                   493
56                   487
52                   484
47                   480
54                   479
46                   475
58                   461
57                   460
53                   459
51                   448
59                   444
55                   443
26                   408
60                   377
25                   357
61                   355
62                   352
63                   269
64                   265


Мы также видим одно непонятное значение в столбце gender. Можно удалить его или просто оставить без изменений - на результат исследования оно не повлияет.

### Шаг 2.3. Изменение типов данных.

Изменим тип данных в столбце total_income с вещественного на целочисленный:

In [12]:
df['total_income'] = df['total_income'].astype(int) 
display(df.head(10))



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.4. Удаление дубликатов.

Приступим к поиску дубликатов в таблице:

In [13]:
display(df['income_type'].value_counts())
display(df['days_employed'].value_counts())
display(df['dob_years'].value_counts())
display(df['education_id'].value_counts())
display(df['family_status'].value_counts())
display(df['gender'].value_counts())
display(df['debt'].value_counts())
display(df['total_income'].value_counts())
display(df['purpose'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

1203.369529    2175
986.927316        1
1893.222792       1
4236.274243       1
6620.396473       1
               ... 
2849.351119       1
5619.328204       1
448.829898        1
1687.038672       1
582.538413        1
Name: days_employed, Length: 19351, dtype: int64

35                   617
40                   609
41                   607
34                   603
38                   598
42                   597
33                   581
39                   573
31                   560
36                   555
44                   547
29                   545
30                   540
48                   538
37                   537
50                   514
43                   513
32                   510
49                   508
28                   503
45                   497
27                   493
56                   487
52                   484
47                   480
54                   479
46                   475
58                   461
57                   460
53                   459
51                   448
59                   444
55                   443
26                   408
60                   377
25                   357
61                   355
62                   352
63                   269
64                   265


1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

0    19784
1     1741
Name: debt, dtype: int64

145017    2175
126262       3
160905       3
150684       3
154199       3
          ... 
109583       1
101387       1
138249       1
280240       1
264193       1
Name: total_income, Length: 18606, dtype: int64

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Из-за разного способа ввода данных наблюдается различное написание одних и тех же слов. 
Для корректировки необходимо узнать все варианты написания категорий в данном столбце.

In [14]:
display(df['education'].unique())

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведем значения в столбце к единому написанию и убедимся, что в столбце осталось всего 5 категорий.

In [15]:
df['education'] = df['education'].str.lower()
display(df['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [16]:
df['education'] = df['education'].str.lower()
display(df['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [28]:
display(df.duplicated().sum()) 

71

In [30]:
df.drop_duplicates(inplace=True)
display(df.duplicated().sum()) 

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим 2 новых датафрейма:

In [17]:
df_1 = pd.DataFrame(df['education_id'],df['education'])
df_2 = pd.DataFrame(df['family_status'],df['family_status_id']) 



Удалим стобцы из основного датасета:

In [18]:
df.drop(['education','family_status'],axis=1)
                    

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем
21521,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость
21523,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

Из-за разного написания в столбце purpose кажется, что целей для оформления кредита очень много. 
На самом деле, их можно объединить в несколько категорий. 

Сначала напишем функцию, определяющую категорию кредитополучателя в зависимости от дохода:

In [19]:
def income_category(income):
    if income<30000:
        return "E"
    if income<50000:
        return "D"
    if income<200000:
        return "C"
    if income<1000000:
        return "B"
    if income>1000001:
        return "A"    


Создадим столбец, используя функцию, заданную выше:

In [20]:
df['total_income_category'] = df['total_income'].apply(income_category)
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

Из-за разного написания в столбце purpose кажется, что целей для оформления кредита очень много. 
На самом деле, их можно объединить в несколько категорий:

In [21]:
    
def purpose_change(purpose):
    if "жил" in purpose or "недв" in purpose:
        return "операции с недвижимостью"
    if "авто" in purpose:
        return "операции с автомобилем"
    if "свад" in purpose:
        return "проведение свадьбы"
    if "обр" in purpose:
        return "получение образования"
        
        
df['purpose_category'] = df['purpose'].apply(purpose_change)
display(df.head(50)) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

### Вопрос 1:

##### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
display(df.groupby('children')['debt'].mean().sort_values(ascending=False))

children
4                        0.097561
2                        0.094404
1                        0.092154
3                        0.081818
0                        0.075129
количество неизвестно    0.073171
5                        0.000000
Name: debt, dtype: float64

##### Вывод 1:Количество задолжавших клиентов действительно растет с количеством детей от 0 до 2, однако, начиная с 3 ребенка, данная закономерность не прослеживается. У клиентов с 5 детьми вообще нет долгов. Вероятно, это связано с тем, что только состоятельные клиенты, способные платить по долгам вовремя, имеют 5 детей.

### Вопрос 2:

##### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
display(df.groupby('family_status')['debt'].mean().sort_values(ascending=False))

family_status
Не женат / не замужем    0.097405
гражданский брак         0.092890
женат / замужем          0.075202
в разводе                0.071130
вдовец / вдова           0.065625
Name: debt, dtype: float64

##### Вывод 2: У людей, не состоящих или не состоявших в браке, процент задолженностей заметно выше. Самый низкий процент - среди вдовцов/вдов.

### Вопрос 3:

##### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
display(df.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = 'mean').sort_values(by = 'debt',ascending=False))

,debt
total_income_category,
E,0.090909
C,0.084915
A,0.080000
B,0.070621
D,0.060000


##### Вывод 3: Уровень дохода не влияет на возврат кредита в срок. При этом, у группы с наименьшими доходами наблюдается самый высокий уровень просрочек по кредитам. Что примечательно, самый низкий уровень задолженностей у предпоследней по доходам группы.

### Вопрос 4:

#### Как разные цели кредита влияют на его возврат в срок?

In [25]:
display(df.groupby(['purpose_category'])['debt'].mean().sort_values(ascending=False))

purpose_category
операции с автомобилем      0.093395
получение образования       0.091994
проведение свадьбы          0.079216
операции с недвижимостью    0.072140
Name: debt, dtype: float64

#### Вывод 4: Среди людей, берущих кредит на операции с недвижимостью, самый низкий процент клиентов с задолженностью. Возможно, это связано с умением лучше планировать свои денежные потоки на много лет вперед и страхом потерять жилье, приобретенное в ипотеку. 

## Общий вывод:

После тщательного изучения данных о заёмщиках можно сделать следующие выводы:

Вывод 1:
Количество задолжавших клиентов действительно растет с количеством детей от 0 до 2, однако, начиная с 3 ребенка, данная закономерность не прослеживается. У клиентов с 5 детьми вообще нет долгов. Вероятно, это связано с тем, что только состоятельные клиенты, способные платить по долгам вовремя, имеют 5 детей.

Вывод 2:
У людей, не состоящих или не состоявших в браке, процент задолженностей заметно выше. Самый низкий процент - среди вдовцов/вдов

Вывод 3: 
Уровень дохода не влияет на возврат кредита в срок

Вывод 4: 
Среди людей, берущих кредит на операции с недвижимостью, самый низкий процент клиентов с задолженностью. 
Возможно, это связано с умением лучше планировать свои денежные потоки на много лет вперед и страхом потерять жилье, приобретенное в ипотеку.